In [1]:
# Data processing
import pandas as pd
import numpy as np
# Data visualization
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.datasets import make_blobs
import seaborn as sns
# Model and performance
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from yellowbrick.cluster import SilhouetteVisualizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBClassifier
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import RFECV 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from sklearn.inspection import permutation_importance
from scipy.cluster.hierarchy import dendrogram, linkage
import scipy.cluster.hierarchy as shc
import itertools
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
ds_df = pd.read_csv('downsized_data.csv')
ds_df

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,days_since_previous_order,total_orders,avg_days_between_orders,reorder_percentage,avg_basket_size,total_reorders
0,2092464,69323,prior,4,0,8,14.0,19057,4,1,Organic Large Extra Fancy Fuji Apple,24,4,14.0,5,22.0,100.0,4.0,111
1,3054441,69323,prior,5,0,17,30.0,45535,12,1,Organic Low Sodium Chicken Broth,69,15,14.0,5,22.0,100.0,12.0,111
2,2250832,186419,prior,86,1,13,7.0,33303,9,1,Organic Hummus,123,4,7.0,87,5.5,100.0,9.0,681
3,679071,186419,prior,11,3,17,9.0,30391,5,1,Organic Cucumber,83,4,7.0,87,5.5,100.0,5.0,681
4,1124370,186419,prior,87,3,14,2.0,9438,4,1,Honey Graham Sticks,61,19,7.0,87,5.5,100.0,4.0,681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71266,3093426,42792,prior,11,5,14,2.0,35951,15,1,Organic Unsweetened Almond Milk,91,16,2.0,11,2.0,100.0,15.0,563
71267,896635,6153,prior,7,5,12,4.0,13176,3,1,Bag of Organic Bananas,24,4,4.0,7,4.0,100.0,3.0,55
71268,8249,135385,prior,27,2,22,9.0,4957,12,1,Total 2% Lowfat Greek Strained Yogurt With Blu...,120,16,9.0,27,9.0,100.0,12.0,414
71269,1845337,26919,prior,19,5,5,19.0,29987,2,1,Shredded Mozzarella,21,16,19.0,19,19.0,100.0,2.0,217


In [3]:
cluster_df = pd.read_csv('cluster_data.csv')
cluster_df

,days_since_previous_order,total_orders,total_reorders,clusters
0,-0.481797,0.531631,-0.804762,1
1,0.273387,0.580959,-0.798223,1
2,0.185172,-1.376511,0.131003,2
3,-0.192420,-1.401175,0.127734,2
4,-0.286818,-1.407341,0.126916,2
...,...,...,...,...
71266,-0.399725,1.026045,0.123119,1
71267,-1.928011,1.530845,0.118284,1
71268,-0.018582,-0.530515,0.021263,2
71269,-0.426890,1.245468,-0.096003,1


In [4]:
# show all columns
pd.set_option('display.max_columns', None)
#Merge cluster DataFrame with ds_df
mb_df = pd.concat([ds_df, cluster_df], axis=1)
mb_df

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,days_since_previous_order,total_orders,avg_days_between_orders,reorder_percentage,avg_basket_size,total_reorders,days_since_previous_order,total_orders,total_reorders,clusters
0,2092464,69323,prior,4,0,8,14.0,19057,4,1,Organic Large Extra Fancy Fuji Apple,24,4,14.0,5,22.0,100.0,4.0,111,-0.481797,0.531631,-0.804762,1
1,3054441,69323,prior,5,0,17,30.0,45535,12,1,Organic Low Sodium Chicken Broth,69,15,14.0,5,22.0,100.0,12.0,111,0.273387,0.580959,-0.798223,1
2,2250832,186419,prior,86,1,13,7.0,33303,9,1,Organic Hummus,123,4,7.0,87,5.5,100.0,9.0,681,0.185172,-1.376511,0.131003,2
3,679071,186419,prior,11,3,17,9.0,30391,5,1,Organic Cucumber,83,4,7.0,87,5.5,100.0,5.0,681,-0.192420,-1.401175,0.127734,2
4,1124370,186419,prior,87,3,14,2.0,9438,4,1,Honey Graham Sticks,61,19,7.0,87,5.5,100.0,4.0,681,-0.286818,-1.407341,0.126916,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71266,3093426,42792,prior,11,5,14,2.0,35951,15,1,Organic Unsweetened Almond Milk,91,16,2.0,11,2.0,100.0,15.0,563,-0.399725,1.026045,0.123119,1
71267,896635,6153,prior,7,5,12,4.0,13176,3,1,Bag of Organic Bananas,24,4,4.0,7,4.0,100.0,3.0,55,-1.928011,1.530845,0.118284,1
71268,8249,135385,prior,27,2,22,9.0,4957,12,1,Total 2% Lowfat Greek Strained Yogurt With Blu...,120,16,9.0,27,9.0,100.0,12.0,414,-0.018582,-0.530515,0.021263,2
71269,1845337,26919,prior,19,5,5,19.0,29987,2,1,Shredded Mozzarella,21,16,19.0,19,19.0,100.0,2.0,217,-0.426890,1.245468,-0.096003,1


In [5]:
#Isolating specific columns for grouping
items_per_transaction = mb_df.groupby(
                        ['user_id', 'clusters', 'order_id'])\
                        ['product_name'].agg(','.join).reset_index()

items_per_transaction

,user_id,clusters,order_id,product_name
0,7,1,23391,Blood Oranges
1,7,1,2452257,Organic Low Fat Chocolate Milk
2,16,1,3192400,Organic Baby Spinach
3,17,1,603534,Gourmet Popping Corn Original
4,17,1,900554,Natural Lime Flavor Sparkling Mineral Water
...,...,...,...,...
69531,206174,0,3304125,Blackberry Cucumber Sparkling Water
69532,206174,2,612512,Egg White Patties
69533,206174,2,1143211,Dha Omega 3 Vitamin D Milk
69534,206208,0,442304,Creme Fraiche French Cultured Cream


In [6]:
#Grouping all transactions by cluster 0 only
items_per_cluster0_transactions = items_per_transaction[items_per_transaction['clusters']==0]
items_per_cluster0_transactions

,user_id,clusters,order_id,product_name
11,27,0,1573122,Organic Blackberries
12,27,0,2138705,Original Turkey Jerky
28,95,0,624228,Organic Yellow Onion
51,160,0,3159678,Fat Free Milk
55,164,0,28065,"Salad Dressing & Dip, Ranch Chia"
...,...,...,...,...
69518,206108,0,2719632,Organic Fuji Apple
69520,206118,0,2362644,Banana
69521,206118,0,3368813,Kosher Dill Pickles
69531,206174,0,3304125,Blackberry Cucumber Sparkling Water


In [7]:
##Grouping all transactions by cluster 1 only
items_per_cluster1_transactions = items_per_transaction[items_per_transaction['clusters']==1]
items_per_cluster1_transactions

,user_id,clusters,order_id,product_name
0,7,1,23391,Blood Oranges
1,7,1,2452257,Organic Low Fat Chocolate Milk
2,16,1,3192400,Organic Baby Spinach
3,17,1,603534,Gourmet Popping Corn Original
4,17,1,900554,Natural Lime Flavor Sparkling Mineral Water
...,...,...,...,...
35614,105162,1,2582133,Cola Soft Drink
35632,105213,1,539977,"Banana, Peach and Mango Fruit Pouch"
35633,105213,1,2582394,Bag of Organic Bananas
35678,105338,1,965137,Pure Irish Butter


In [8]:
#Grouping all transactions by cluster 2 only
items_per_cluster2_transactions = items_per_transaction[items_per_transaction['clusters']==2]
items_per_cluster2_transactions

,user_id,clusters,order_id,product_name
34240,100953,2,1005233,Sustainably Soft Bath Tissue
34329,101188,2,1586044,Seedless Cucumbers
34460,101607,2,464749,Organic Garnet Sweet Potato (Yam)
34469,101634,2,245241,Sparkling Water
34493,101719,2,82280,Unsweetened Almondmilk
...,...,...,...,...
69529,206154,2,3103776,Lemon & Chive Dressing
69530,206155,2,2473813,Shredded Hash Browns
69532,206174,2,612512,Egg White Patties
69533,206174,2,1143211,Dha Omega 3 Vitamin D Milk


In [9]:
items_per_transaction.to_csv(r'C:\Users\p-mir\MS Data Science\Big Data Technologies\Final\items_per_transaction_data.csv', index=False)
items_per_cluster0_transactions.to_csv(r'C:\Users\p-mir\MS Data Science\Big Data Technologies\Final\items_per_cluster0_transactions.csv', index=False)
items_per_cluster1_transactions.to_csv(r'C:\Users\p-mir\MS Data Science\Big Data Technologies\Final\items_per_cluster1_transactions.csv', index=False)
items_per_cluster2_transactions.to_csv(r'C:\Users\p-mir\MS Data Science\Big Data Technologies\Final\items_per_cluster2_transactions.csv', index=False)

In [10]:
#Creating an Item List for ALL Clusters 
item_list = [items.split(',') for items in items_per_transaction['product_name']]
item_list_df = pd.DataFrame({'Items': item_list})

item_list_df

,Items
0,[Blood Oranges]
1,[Organic Low Fat Chocolate Milk]
2,[Organic Baby Spinach]
3,[Gourmet Popping Corn Original]
4,[Natural Lime Flavor Sparkling Mineral Water]
...,...
69531,[Blackberry Cucumber Sparkling Water]
69532,[Egg White Patties]
69533,[Dha Omega 3 Vitamin D Milk]
69534,[Creme Fraiche French Cultured Cream]


In [67]:
item_list

[['Blood Oranges'],
 ['Organic Low Fat Chocolate Milk'],
 ['Organic Baby Spinach'],
 ['Gourmet Popping Corn Original'],
 ['Natural Lime Flavor Sparkling Mineral Water'],
 ['Tender white meat turkey',
  ' pasta',
  ' mushrooms and celery in a creamy sauce Turkey Tetrazzini'],
 ['Grade A Extra Large Eggs'],
 ['Vinegar', ' Apple Cider'],
 ['Shredded Sharp Cheddar Cheese'],
 ['Cheez-It Baked Snack Crackers'],
 ['Crunchy Rice Rollers'],
 ['Organic Blackberries'],
 ['Original Turkey Jerky'],
 ['Salt & Pepper Krinkle Chips'],
 ['Italian Sausage'],
 ['Original 0 Calorie Sweetener Packets 115 Count'],
 ['Plain Goat Milk Yogurt'],
 ['Yobaby Organic Plain Yogurt'],
 ['Natural Artesian Bottled Water'],
 ['Coconut Almond Minis Frozen Dessert Bars'],
 ['Organic Polenta'],
 ['Gluten Free Sliced Mountain White Bread'],
 ['Organic Half & Half'],
 ['Sensitive Facial Cleansing Towelettes'],
 ['Gluten Free Peanut Butter Berry  Chewy With A Crunch Granola Bars'],
 ['Gluten Free Honey Almond Granola'],
 ['R

In [12]:
#Creating an Item List for Cluster 0 only
item_list0 = [items.split(',') for items in items_per_cluster0_transactions['product_name']]
item_list0_df = pd.DataFrame({'Items': item_list0})

item_list0_df

,Items
0,[Organic Blackberries]
1,[Original Turkey Jerky]
2,[Organic Yellow Onion]
3,[Fat Free Milk]
4,"[Salad Dressing & Dip, Ranch Chia]"
...,...
16782,[Organic Fuji Apple]
16783,[Banana]
16784,[Kosher Dill Pickles]
16785,[Blackberry Cucumber Sparkling Water]


In [13]:
#Creating an Item List for Cluster 1 Only 
item_list1 = [items.split(',') for items in items_per_cluster1_transactions['product_name']]
item_list1_df = pd.DataFrame({'Items': item_list1})

item_list1_df

,Items
0,[Blood Oranges]
1,[Organic Low Fat Chocolate Milk]
2,[Organic Baby Spinach]
3,[Gourmet Popping Corn Original]
4,[Natural Lime Flavor Sparkling Mineral Water]
...,...
26667,[Cola Soft Drink]
26668,"[Banana, Peach and Mango Fruit Pouch]"
26669,[Bag of Organic Bananas]
26670,[Pure Irish Butter]


In [14]:
#Creating an Item List for Cluster 2 Only 
item_list2 = [items.split(',') for items in items_per_cluster2_transactions['product_name']]
item_list2_df = pd.DataFrame({'Items': item_list2})

item_list2_df

,Items
0,[Sustainably Soft Bath Tissue]
1,[Seedless Cucumbers]
2,[Organic Garnet Sweet Potato (Yam)]
3,[Sparkling Water]
4,[Unsweetened Almondmilk]
...,...
26072,[Lemon & Chive Dressing]
26073,[Shredded Hash Browns]
26074,[Egg White Patties]
26075,[Dha Omega 3 Vitamin D Milk]


In [15]:
#Saving item_list arrays to file
np.savetxt(r'C:\Users\p-mir\MS Data Science\Big Data Technologies\Final\item_list.csv', item_list, delimiter=',', fmt='%s', encoding='utf-8')
np.savetxt(r'C:\Users\p-mir\MS Data Science\Big Data Technologies\Final\item_list0.csv', item_list0, delimiter=',', fmt='%s', encoding='utf-8')
np.savetxt(r'C:\Users\p-mir\MS Data Science\Big Data Technologies\Final\item_list1.csv', item_list1, delimiter=',', fmt='%s', encoding='utf-8')
np.savetxt(r'C:\Users\p-mir\MS Data Science\Big Data Technologies\Final\item_list2.csv', item_list2, delimiter=',', fmt='%s', encoding='utf-8')

C:\Users\p-mir\anaconda3\lib\site-packages\numpy\lib\npyio.py:1378: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.asarray(X)
